In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging
import sys
import time, datetime
from typing import Dict, Iterable
import os
import gzip
import json

from google.colab import drive

# Mount Google Drive to the current Colab session
drive.mount('/content/drive')

# Verify that your drive is accessible
!ls /content/drive/MyDrive

Mounted at /content/drive
'16 б.gdoc'			  deleteme.zip		   share
'6 фото.     5 лет блондин.JPG'   futuristic-office.jpeg  'Tanya Passport.JPG'
'Colab Notebooks'		  rental_houses.gsheet	  'лечение травами 3.gdoc'


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text

In [4]:
def write_jsonl(filename: str, data: Iterable[Dict], append: bool = False):
    """
    Writes an iterable of dictionaries to jsonl
    """
    if append:
        mode = 'ab'
    else:
        mode = 'wb'
    filename = os.path.expanduser(filename)
    if filename.endswith(".gz"):
        with open(filename, mode) as fp:
            with gzip.GzipFile(fileobj=fp, mode='wb') as gzfp:
                for x in data:
                    gzfp.write((json.dumps(x) + "\n").encode('utf-8'))
    else:
        with open(filename, mode) as fp:
            for x in data:
                fp.write((json.dumps(x) + "\n").encode('utf-8'))


def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, 'rt') as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    yield json.loads(line)


def read_problems(evalset_file: str) -> Dict[str, Dict]:
    return {task["task_id"]: task for task in stream_jsonl(evalset_file)}

In [5]:
def is_full_function(input_str: str) -> str:
    '''
    Determine if input string is a Python function
    using simple heuristics (all that's needed)
    '''
    lines = input_str.splitlines()
    for line in lines:
        if line.startswith('def '):
            return True
    return False


def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def add_import(text: str) -> str:
    ''' Adds import statements stripped by LLM '''
    text = text.strip()
    s = 'from typing import List'
    if 'def ' in text and s not in text:
        text = s + '\n\n' + text
    return text

string = 'def my_func(input):\n    return output'
print(add_import(string))

from typing import List

def my_func(input):
    return output


In [6]:
# read dataset
file = '/content/drive/My Drive/Colab Notebooks/data/Big_Code_Bench_Test.jsonl.gz'
tasks = read_problems(file)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 5:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: BigCodeBench/0 <class 'str'>

task_id:
BigCodeBench/0

test:


import unittest
from unittest.mock import patch
from random import seed, shuffle
import itertools
class TestCases(unittest.TestCase):
    def test_default_numbers(self):
        # Test with default number range (1 to 10) to check that the result is a positive float.
        result = task_func()
        self.assertIsInstance(result, float)
        self.assertGreater(result, 0)
    def test_custom_list(self):
        # Test with a custom list of small positive integers to ensure proper handling and positive result.
        result = task_func([1, 2, 3])
        self.assertIsInstance(result, float)
        self.assertGreater(result, 0)
    def test_negative_numbers(self):
        # Test with negative numbers to verify the function handles and returns a positive result.
        result = task_func([-3, -2, -1])
        self.assertIsInstance(result, float)
        self.assertGreater(result, 

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "NTQAI/Nxcode-CQ-7B-orpo",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("NTQAI/Nxcode-CQ-7B-orpo")
print(f'Model device: {model.device}')

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/7.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

Model device: cuda:0


In [8]:
print(f'Model temperature: {model.config.temperature}. Model top_p: {model.config.top_p}')

Model temperature: 1.0. Model top_p: 1.0


In [28]:
def generate_response(prompt):
  messages = [
      {"role": "user", "content": prompt}
  ]
  inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
  outputs = model.generate(inputs, max_new_tokens=512, do_sample=True, top_k=50, top_p=1.0, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
  res = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
  return res

In [13]:
task = tasks['BigCodeBench/15']

# smaller models ouput additional trash with this propmt (before and after the code)
completion_prompt = 'Complete the following Python code:\n'

completion_prompt = '''1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:\n\n'''
print(completion_prompt + task['prompt'])

1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import subprocess
import csv
import os

def task_func(commands_file_path, output_dir_path):
    """
    Execute a list of shell commands read from a CSV file and save the outputs in separate files.
    Each command's output is written to a unique file in the specified output directory.
    If a command fails, the error message along with the exit code is appended to the respective output file.

In [14]:
# to save results
model_nickname = 'nxcode'
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
file_name      = f'/content/drive/My Drive/Colab Notebooks/logs/{model_nickname}_samples_{time_stamp}.jsonl'
file_name

'/content/drive/My Drive/Colab Notebooks/logs/nxcode_samples_20250121_092801_7325.jsonl'

In [26]:
def get_import_statements(code_str: str) -> str:
    """
    Keeps lines that begin with 'import' or 'from' from a multiline string.
    """
    lines = code_str.splitlines()
    filtered_lines = []
    for line in lines:
        stripped_line = line.strip()
        if stripped_line.startswith('import ') or stripped_line.startswith('from '):
            filtered_lines.append(line)
    res = '\n'.join(filtered_lines) + '\n'
    s = 'from typing import List'
    if s not in res:
        res = s + '\n' + res
    return res

# Example usage
code_with_imports = '''import collections
import random
import string

def task_func(length=100):
    return 1
'''
print( get_import_statements(code_with_imports) )

from typing import List
import collections
import random
import string



In [ ]:
num_samples_per_task = 1
completions = []
is_full_func = []

for task_id, task_body in tasks.items():
    print(task_id)
    #if task_id == 'BigCodeBench/7':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = completion_prompt + task_body['prompt']
        try:
            completion  = generate_response( full_prompt )
        except Exception as e:
            completion = f"Error generating a completion:\n{e}"

        completion = remove_triple_backticks_at_edges(completion.strip())
        is_full = is_full_function(completion)
        is_full_func.append(is_full)
        if is_full:
            completion = get_import_statements(task_body['prompt']) + '\n' + completion
        completions.append( {'task_id': task_id, 'completion': completion} )
        write_jsonl(file_name, completions)
        print('Prompt:\n', full_prompt, '\n', '-'*75, '\n', sep='')
        print('Completion:\n', completion, sep='')
        print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n", '\n', '='*75, '\n', sep='')

BigCodeBench/0
Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import itertools
from random import shuffle

def task_func(numbers=list(range(1, 3))):
    """
    Calculates the average of the sums of absolute differences between each pair of consecutive numbers 
    for all permutations of a given list. Each permutation is shuffled before calculating the differences.

    Args:
    - numbers (list): A list of numbers. Default is num

In [ ]:
# count how many full funcs were generated vs. only func bodies without func signatures
from collections import Counter
Counter(is_full_func)

In [ ]:
from google.colab import runtime
runtime.unassign()